In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.ticker import FuncFormatter
from ipywidgets import Output

In [2]:


# API Key from Alpha Vantage
# R4BTXGGSVD9HT6GU
# OZ4VEG5LX8WR3NFK
# K2Y37FKIZBOPQ9EQ
api_key = 'K2Y37FKIZBOPQ9EQ'

# List of company symbols
symbols = ['ACN', 'CTSH', 'IBM', 'INFY', 'WIT', 'BAH']

# Initialize an empty list to store the data
data_list = []

# Loop through each symbol and fetch the annual income statements
for symbol in symbols:
    url_Alpha = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}'
    response = requests.get(url_Alpha)
    data = response.json()
    
    # Check if there are annual reports and add their data to the list
    if 'annualReports' in data:
        for report in data['annualReports']:
            report['symbol'] = symbol  # Add the company symbol to the report
            data_list.append(report)

# Create a DataFrame from the list of data
df = pd.DataFrame(data_list)



In [3]:
# Reorganicing columns in df

columns = ['symbol'] + [col for col in df.columns if col != 'symbol']
df = df[columns]

In [4]:
# Configure pandas to display a large number of columns
pd.set_option('display.max_columns', None)  # None means no maximum value
pd.set_option('display.expand_frame_repr', False)  # Prevent DataFrame repr from wrapping into multiple lines
print(df)

   symbol fiscalDateEnding reportedCurrency  grossProfit totalRevenue costOfRevenue costofGoodsAndServicesSold operatingIncome sellingGeneralAndAdministrative researchAndDevelopment operatingExpenses investmentIncomeNet netInterestIncome interestIncome interestExpense nonInterestIncome otherNonOperatingIncome depreciation depreciationAndAmortization incomeBeforeTax incomeTaxExpense interestAndDebtExpense netIncomeFromContinuingOperations comprehensiveIncomeNetOfTax        ebit      ebitda   netIncome
0     ACN       2023-08-31              USD   8356684000  64111745000   55755061000                43380138000      8809889000                     10858572000             1298657000       13673580000           280409000         -47525000      232884000        47525000        -135586000                96559000    620659000                   453205000      9007359000       2135802000               47525000                        7003530000                  7318798000  9054884000  9508089000 

In [5]:
# DataFrame filter to obtein rows "reportedCurrency" is "INR"
df_inr = df[df['reportedCurrency'] == 'INR']

# unique dates from "fiscalDateEnding"
Unique_dates = df_inr['fiscalDateEnding'].unique().tolist()


In [6]:
# API KEY openexchangerates
api_key = '87bf2801524b4db68c05bb810726d141'

# dates
fechas = Unique_dates

# Initialize an empty list to store the data
exchange_dict = {}

# API´s URL 
for fecha in fechas:
    url_Alpha = f'https://openexchangerates.org/api/historical/{fecha}.json?app_id={api_key}&symbols=INR&base=USD'
    response = requests.get(url_Alpha)
    fx = response.json()
    exchange_dict[fecha] = fx['rates']['INR']



In [7]:
# Creating a df with exchange rate info

df['exchange_rate'] = None
for index, row in df.iterrows():
    # Verify condition:
    if row['reportedCurrency'] == 'INR' and row['fiscalDateEnding'] in exchange_dict:
        # update fx
        df.at[index, 'exchange_rate'] = exchange_dict[row['fiscalDateEnding']]


In [8]:
# Columns tranformation

columns_to_convert = df.columns.to_list()
columns_to_convert.remove('symbol')
columns_to_convert.remove('fiscalDateEnding')
columns_to_convert.remove('reportedCurrency')

for columna in columns_to_convert:
    df[columna] = pd.to_numeric(df[columna], errors='coerce')

In [9]:
# Export data to excel to check if everything is ok 

df.to_excel('consulting_companies.xlsx', index=False)

In [ ]:
# Removes 'exchange_rate' from the list of columns that we plan to convert.
columns_to_convert.remove('exchange_rate')


In [ ]:
# Copy the original DataFrame to keep the conversion separate.
df_USD = df

# Create a mask for rows with 'INR' as the reported currency.
mask = df_USD['reportedCurrency']=='INR'

# Convert selected columns from INR to USD for these rows.
df_USD.loc[mask, columns_to_convert] = df_USD.loc[mask, columns_to_convert].div(df['exchange_rate'], axis=0)


In [ ]:
# Update the 'reportedCurrency' column to 'USD' where it was 'INR'.
df_USD.loc[df_USD['reportedCurrency'] == 'INR', 'reportedCurrency'] = 'USD'


In [ ]:
# Adjust pandas display settings for better DataFrame visualization.
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', None)  
pd.set_option('display.float_format', lambda x: '%.2f' % x)


In [ ]:
# Display the first 100 rows of the DataFrame.
df_USD.head(100)


In [ ]:
# Export the DataFrame to an Excel file for external use.
df_USD.to_excel('consulting_companies_USD.xlsx', index=False)


In [20]:
# Filter, convert, and sort data for a specific company ('ACN').
df_acn = df_USD[df_USD['symbol'] == 'ACN']
df_USD.loc[df_USD['symbol'] == 'ACN', 'fiscalDateEnding'] = pd.to_datetime(df_USD[df_USD['symbol'] == 'ACN']['fiscalDateEnding'])
df_acn = df_USD[df_USD['symbol'] == 'ACN'].sort_values(by='fiscalDateEnding')

# Print the final DataFrame for the company.
print(df_acn)


   symbol     fiscalDateEnding reportedCurrency    grossProfit   totalRevenue  costOfRevenue  costofGoodsAndServicesSold  operatingIncome  sellingGeneralAndAdministrative  researchAndDevelopment  operatingExpenses  investmentIncomeNet  netInterestIncome  interestIncome  interestExpense  nonInterestIncome  otherNonOperatingIncome  depreciation  depreciationAndAmortization  incomeBeforeTax  incomeTaxExpense  interestAndDebtExpense  netIncomeFromContinuingOperations  comprehensiveIncomeNetOfTax          ebit        ebitda     netIncome  exchange_rate
13    ACN  2010-08-31 00:00:00              USD  7250747000.00 23094078000.00 15843331000.00              15843331000.00    2914845000.00                    4326364000.00            376985000.00      4724741000.00          29931000.00       -14677000.00             NaN      14677000.00       -30741000.00             -15724000.00  269072000.00                 474688000.00    2634566000.00      853910000.00             14677000.00              